<a href="https://colab.research.google.com/github/Worachet-Ch/data_viz_2022/blob/main/random_NaN_by_Probability_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
import numpy as np
from sklearn.impute import SimpleImputer
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install missingpy
import missingpy
from missingpy import MissForest
from sklearn.metrics import mean_squared_error
import time
from sklearn import neighbors
from sklearn.impute import KNNImputer
!pip install impyute
import impyute as impy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [76]:
import pandas as pd
import os
from google.colab import drive #เชื่อม Google drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/project' #เข้า drive
data_file_path = os.path.join(path,'dataset2.csv') #ไปที่ไฟล์ confirmed-cases-since-280265.csv
data = pd.read_csv(data_file_path) #อ่านไฟล์ csv

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data

In [77]:
data

,No.,LabNo,Sex,AgeSYear,Enose,Overall,Body shape age,Kidney,Heart,Liver,...,071_HbA1c,CK,LD-P,AMY7,TP1,DBIL,TBIL,ALB1,GGT,homocysteine
0,1.0,NaN,2,41.0,NaN,NaN,NaN,NaN,NaN,NaN,...,4.8,79.0,231.0,59.0,7.16,0.10,0.30,4.48,11.0,10.88
1,2.0,201807163166.00,1,49.0,NaN,1.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.11
2,3.0,201807163167,1,44.0,NaN,NaN,NaN,NaN,NaN,NaN,...,4.6,85.0,175.0,104.0,7.43,0.15,1.26,4.70,15.0,11.94
3,4.0,201807163168,2,37.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.01
4,5.0,201807163169,2,38.0,NaN,NaN,NaN,NaN,NaN,NaN,...,5.4,57.0,177.0,89.0,7.11,0.02,0.20,3.92,24.0,8.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2269,NaN,V17,2,31.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2270,NaN,V19,2,32.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2271,NaN,V20,1,28.0,NaN,1.0,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2272,NaN,V21,1,29.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
data.dropna(subset=['telomere length (kb) '], inplace = True)
data.drop(['No.', 'LabNo','DHEAs (BRIA)'], axis=1,inplace = True)
data.drop(data.columns[2: 10], axis=1,inplace = True)
data.drop(data.columns[27:37], axis=1,inplace = True)
data.drop('telomere length (kb) ',1,inplace = True)

<ipython-input-78-1a7942661c33>:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data.drop('telomere length (kb) ',1,inplace = True)


In [79]:
data

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,003_Creatinine,004_Uric Acid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,013_Alkaline Phos,071_HbA1c
49,2,58.0,47.2,156.0,19.4,112.0,64.0,70.0,NaN,NaN,...,0.6,4.8,180.0,97.0,62.0,98.0,25.0,32.0,40.0,5.8
56,2,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
85,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
103,2,62.0,54.4,165.0,20.0,95.0,75.0,83.0,21.6,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
107,2,86.0,49.0,154.0,20.7,129.0,79.0,83.0,NaN,NaN,...,0.7,5.4,183.0,86.0,39.0,127.0,12.0,22.0,82.0,6.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2267,2,25.0,47.0,165.0,17.3,104.0,63.0,89.0,NaN,NaN,...,0.6,4.8,232.0,36.0,96.0,128.0,18.0,12.0,46.0,NaN
2269,2,31.0,72.2,168.0,25.6,123.0,89.0,69.0,NaN,NaN,...,0.6,6.1,187.0,49.0,51.0,127.0,21.0,24.0,71.0,NaN
2270,2,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.7,5.1,222.0,58.0,79.0,132.0,17.0,10.0,34.0,NaN
2272,1,29.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,6.8,140.0,92.0,48.0,73.0,31.0,75.0,69.0,NaN


In [80]:
P = 0
p1 = 0
list_mis = []
for i in data:
  if data[i].isna().sum() == 0 :
    continue
  else:
    ms = data[i].isna().sum()
    P = (ms/1734).round(14)
    p1 = p1 + P
    print(f'ความน่าจะเป็นของ {i} = {P}')
    list_mis.append(P)
print('****************************************')
print(f'ความน่าจะเป็นทั้งหมด = {p1}')

ความน่าจะเป็นของ Weight = 0.00461361014994
ความน่าจะเป็นของ Height = 0.00461361014994
ความน่าจะเป็นของ BMI = 0.00461361014994
ความน่าจะเป็นของ Systolic = 0.00461361014994
ความน่าจะเป็นของ Diastolic = 0.00519031141869
ความน่าจะเป็นของ HeartRate = 0.02941176470588
ความน่าจะเป็นของ SMM = 0.12975778546713
ความน่าจะเป็นของ Fat Mass = 0.12975778546713
ความน่าจะเป็นของ % Body fat = 0.12975778546713
ความน่าจะเป็นของ waist to hip radio = 0.12975778546713
ความน่าจะเป็นของ abdominal circumference = 0.13033448673587
ความน่าจะเป็นของ visceral fat area = 0.12975778546713
ความน่าจะเป็นของ 001_Glucose = 0.00057670126874
ความน่าจะเป็นของ 004_Uric Acid = 0.05363321799308
ความน่าจะเป็นของ 012_ALT = 0.00057670126874
ความน่าจะเป็นของ 013_Alkaline Phos = 0.01672433679354
ความน่าจะเป็นของ 071_HbA1c = 0.09630911188005
****************************************
ความน่าจะเป็นทั้งหมด = 1.0


In [81]:
data.rename(columns={'Fat Mass':'FatMass','% Body fat':'Bodyfat','waist to hip radio':'waisttohip','abdominal circumference':'abdominal',
                    'visceral fat area':'visceral','004_Uric Acid':'UricAcid',''
                                'HDL-C':'HDL','013_Alkaline Phos':'Alkaline_Phos'},inplace = True)

In [82]:
data_file_path1 = os.path.join(path,'data_notmissing.csv') #ไปที่ไฟล์ confirmed-cases-since-280265.csv
data1 = pd.read_csv(data_file_path1) #อ่านไฟล์ csv

In [83]:
data1.drop({'Unnamed: 0','telomere length (kb) '},1,inplace = True)

<ipython-input-83-0e7a2b46e4dd>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data1.drop({'Unnamed: 0','telomere length (kb) '},1,inplace = True)


In [84]:
data1.rename(columns={'Fat Mass':'FatMass','% Body fat':'Bodyfat','waist to hip radio':'waisttohip','abdominal circumference':'abdominal',
                    'visceral fat area':'visceral','004_Uric Acid':'UricAcid',''
                                'HDL-C':'HDL','013_Alkaline Phos':'Alkaline_Phos'},inplace = True)

In [85]:
data1

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,FatMass,...,003_Creatinine,UricAcid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,Alkaline_Phos,071_HbA1c
0,2,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2,62.0,54.4,165.0,20.0,95.0,75.0,83.0,21.6,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.5,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1,46.0,72.3,170.0,25.0,133.0,77.0,107.0,29.2,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,72.0,170.0,24.9,131.0,74.0,69.0,30.5,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,27.8,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.9,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2,61.0,61.8,159.0,24.4,117.0,60.0,72.0,23.4,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


In [86]:
list1 = []
for i in data:
  if data[i].isna().sum() == 0:
    continue
  else: 
    list1.append(i)
list1

['Weight',
 'Height',
 'BMI',
 'Systolic',
 'Diastolic',
 'HeartRate',
 'SMM',
 'FatMass',
 'Bodyfat',
 'waisttohip',
 'abdominal',
 'visceral',
 '001_Glucose',
 'UricAcid',
 '012_ALT',
 'Alkaline_Phos',
 '071_HbA1c']

In [87]:
list_mis

[0.00461361014994,
 0.00461361014994,
 0.00461361014994,
 0.00461361014994,
 0.00519031141869,
 0.02941176470588,
 0.12975778546713,
 0.12975778546713,
 0.12975778546713,
 0.12975778546713,
 0.13033448673587,
 0.12975778546713,
 0.00057670126874,
 0.05363321799308,
 0.00057670126874,
 0.01672433679354,
 0.09630911188005]

# 10 %

## แทนค่า NaN

In [88]:
random_column = np.random.choice(list1,int((153*17)*(10/100)),p=list_mis)
random_column = list(random_column)
random_column

['abdominal',
 'UricAcid',
 'SMM',
 '071_HbA1c',
 'waisttohip',
 'UricAcid',
 'FatMass',
 'abdominal',
 'Bodyfat',
 'Bodyfat',
 'SMM',
 'FatMass',
 'visceral',
 'waisttohip',
 'Bodyfat',
 'UricAcid',
 '071_HbA1c',
 'abdominal',
 'HeartRate',
 'Bodyfat',
 'waisttohip',
 'SMM',
 '071_HbA1c',
 'Bodyfat',
 'waisttohip',
 'SMM',
 'abdominal',
 'Bodyfat',
 'UricAcid',
 'SMM',
 'HeartRate',
 'waisttohip',
 'waisttohip',
 'Alkaline_Phos',
 'FatMass',
 'waisttohip',
 'abdominal',
 'visceral',
 'SMM',
 'visceral',
 'Bodyfat',
 'Bodyfat',
 'FatMass',
 'abdominal',
 'SMM',
 'SMM',
 'abdominal',
 'FatMass',
 'SMM',
 'SMM',
 '071_HbA1c',
 'UricAcid',
 'visceral',
 'FatMass',
 'waisttohip',
 'waisttohip',
 'FatMass',
 'Bodyfat',
 'abdominal',
 '071_HbA1c',
 'FatMass',
 'abdominal',
 '071_HbA1c',
 'abdominal',
 '071_HbA1c',
 'Bodyfat',
 'abdominal',
 'SMM',
 'abdominal',
 'visceral',
 'FatMass',
 'FatMass',
 'Bodyfat',
 'SMM',
 'FatMass',
 'UricAcid',
 'SMM',
 'visceral',
 'Height',
 'FatMass',
 'Body

In [89]:
weight = []
height = []
bmi = []
systolic = []
diastolic = []
heartrate = []
smm = []
fatmass = []
bodyfat = []
wait = []
abdominal = []
visceral = []
glucose = []
uric_acid = []
ALT = []
Alkaline = []
HbA1c = []

In [90]:
for i in random_column:
  if i == 'Weight':
    weight.append(i)  
  elif i == 'Height':
    height.append(i)
  elif i == 'BMI':
    bmi.append(i)
  elif i == 'Systolic':
    systolic.append(i)
  elif i == 'Diastolic':
    diastolic.append(i)
  elif i == 'HeartRate':
    heartrate.append(i)
  elif i == 'SMM':
    smm.append(i)
  elif i == 'FatMass':
    fatmass.append(i)
  elif i == 'Bodyfat':
    bodyfat.append(i)
  elif i == 'waisttohip':
    wait.append(i)
  elif i == 'abdominal':
    abdominal.append(i)
  elif i == 'visceral':
    visceral.append(i)
  elif i == '001_Glucose':
    glucose.append(i)
  elif i == 'UricAcid':
    uric_acid.append(i)
  elif i == '012_ALT':
    ALT.append(i)
  elif i == 'Alkaline_Phos':
    Alkaline.append(i)
  elif i == '071_HbA1c':
    HbA1c.append(i)

In [91]:
list2 = [len(weight),len(height),len(bmi),len(systolic),len(diastolic),len(heartrate),len(smm),
         len(fatmass),len(bodyfat),len(wait),len(abdominal),len(visceral),len(glucose),len(uric_acid),
         len(ALT),len(Alkaline),len(HbA1c)]
list3 = []
list4 = []

for i in range(153):
  m = 1/153
  list3.append(i)
  list4.append(m)

In [92]:
list2

[1, 2, 2, 1, 1, 7, 35, 33, 40, 28, 42, 33, 0, 11, 0, 2, 22]

In [93]:
list1

['Weight',
 'Height',
 'BMI',
 'Systolic',
 'Diastolic',
 'HeartRate',
 'SMM',
 'FatMass',
 'Bodyfat',
 'waisttohip',
 'abdominal',
 'visceral',
 '001_Glucose',
 'UricAcid',
 '012_ALT',
 'Alkaline_Phos',
 '071_HbA1c']

In [94]:
dt10 = data1.copy()
dt10

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,FatMass,...,003_Creatinine,UricAcid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,Alkaline_Phos,071_HbA1c
0,2,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2,62.0,54.4,165.0,20.0,95.0,75.0,83.0,21.6,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.5,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1,46.0,72.3,170.0,25.0,133.0,77.0,107.0,29.2,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,72.0,170.0,24.9,131.0,74.0,69.0,30.5,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,27.8,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.9,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2,61.0,61.8,159.0,24.4,117.0,60.0,72.0,23.4,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


In [95]:
for i,j in zip(list1,list2):
  dt10[i].ravel()[np.random.choice(list3,j,p= list4, replace=False)] = np.nan

In [96]:
dt10.isna().sum()

Sex                  0
AgeSYear             0
Weight               1
Height               2
BMI                  2
Systolic             1
Diastolic            1
HeartRate            7
SMM                 35
FatMass             33
Bodyfat             40
waisttohip          28
abdominal           42
visceral            33
001_Glucose          0
002_BUN              0
003_Creatinine       0
UricAcid            11
005_Cholesterol      0
006_Triglyceride     0
008_HDL-C            0
009_LDL              0
010_AST              0
012_ALT              0
Alkaline_Phos        2
071_HbA1c           22
dtype: int64

In [97]:
dt10

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,FatMass,...,003_Creatinine,UricAcid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,Alkaline_Phos,071_HbA1c
0,2,57.0,47.4,153.0,20.2,114.0,75.0,69.0,NaN,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,NaN,NaN,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2,62.0,54.4,165.0,20.0,95.0,75.0,83.0,21.6,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.5,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1,46.0,72.3,170.0,NaN,133.0,77.0,107.0,NaN,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,72.0,170.0,24.9,131.0,74.0,69.0,30.5,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,NaN
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,27.8,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.9,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,NaN
151,2,61.0,61.8,159.0,24.4,117.0,60.0,72.0,23.4,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,NaN


## Imputation

### MEAN

In [98]:
from sklearn.impute import SimpleImputer

In [99]:
t_start = time.time()
mean_imputer = SimpleImputer(strategy='mean')
data_MEAN10 = mean_imputer.fit_transform(dt10.copy())
t_end = time.time()
t_MEAN10 = (t_end- t_start)

In [100]:
data_MEAN10 = pd.DataFrame(data_MEAN10)
data_MEAN10 = data_MEAN10.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})
data_MEAN10

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,57.0,47.4,153.0,20.200000,114.0,75.0,69.0,23.042373,12.500000,...,1.0,5.00000,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.100000
1,2.0,24.0,53.2,157.0,21.600000,103.0,69.0,78.0,23.042373,14.726667,...,0.8,5.20000,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.400000
2,2.0,62.0,54.4,165.0,20.000000,95.0,75.0,83.0,21.600000,15.200000,...,0.7,4.10000,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.600000
3,2.0,35.0,56.3,161.0,21.700000,95.0,57.0,72.0,22.500000,15.400000,...,0.6,5.00000,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.100000
4,1.0,46.0,72.3,170.0,21.422517,133.0,77.0,107.0,23.042373,19.000000,...,0.9,6.90000,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1.0,60.0,72.0,170.0,24.900000,131.0,74.0,69.0,30.500000,13.800000,...,0.9,6.90000,182.0,179.0,39.0,107.0,36.0,31.0,79.0,5.563359
149,1.0,58.0,62.0,168.0,22.000000,103.0,76.0,72.0,27.800000,11.900000,...,1.0,6.80000,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.800000
150,2.0,42.0,55.5,165.0,20.400000,102.0,52.0,77.0,22.900000,14.100000,...,0.8,4.70000,212.0,68.0,63.0,135.0,11.0,16.0,33.0,5.563359
151,2.0,61.0,61.8,159.0,24.400000,117.0,60.0,72.0,23.400000,19.200000,...,0.8,5.10000,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.563359


In [101]:
mse_MEAN10 = mean_squared_error(data1,data_MEAN10)
mse_MEAN10

14.794385185765863

### MISS Forest

In [102]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install missingpy
import missingpy
from missingpy import MissForest
from sklearn.metrics import mean_squared_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
t_start = time.time()
MF = MissForest()
data_MF10 = MF.fit_transform(dt10.copy())
t_end = time.time()
t_MF10= (t_end- t_start)

In [104]:
data_MF10 = pd.DataFrame(data_MF10)

In [105]:
data_MF10 = data_MF10.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})
data_MF10

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,57.0,47.4,153.0,20.200,114.0,75.0,69.0,19.466,12.500,...,1.0,5.000,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.100
1,2.0,24.0,53.2,157.0,21.600,103.0,69.0,78.0,21.346,15.333,...,0.8,5.200,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.400
2,2.0,62.0,54.4,165.0,20.000,95.0,75.0,83.0,21.600,15.200,...,0.7,4.100,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.600
3,2.0,35.0,56.3,161.0,21.700,95.0,57.0,72.0,22.500,15.400,...,0.6,5.000,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.100
4,1.0,46.0,72.3,170.0,23.732,133.0,77.0,107.0,30.921,19.000,...,0.9,6.900,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1.0,60.0,72.0,170.0,24.900,131.0,74.0,69.0,30.500,13.800,...,0.9,6.900,182.0,179.0,39.0,107.0,36.0,31.0,79.0,5.633
149,1.0,58.0,62.0,168.0,22.000,103.0,76.0,72.0,27.800,11.900,...,1.0,6.800,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.800
150,2.0,42.0,55.5,165.0,20.400,102.0,52.0,77.0,22.900,14.100,...,0.8,4.700,212.0,68.0,63.0,135.0,11.0,16.0,33.0,5.533
151,2.0,61.0,61.8,159.0,24.400,117.0,60.0,72.0,23.400,19.200,...,0.8,5.100,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.150


In [106]:
mse_MF10 = mean_squared_error(data1,data_MF10)
mse_MF10

2.3144751588210157

In [107]:
data_MF10.isna().sum()

Sex                        0
AgeSYear                   0
Weight                     0
Height                     0
BMI                        0
Systolic                   0
Diastolic                  0
HeartRate                  0
SMM                        0
Fat Mass                   0
% Body fat                 0
waist to hip radio         0
abdominal circumference    0
visceral fat area          0
Glucose                    0
BUN                        0
Creatinine                 0
Uric Acid                  0
Cholesterol                0
Triglyceride               0
HDL-C                      0
LDL                        0
AST                        0
ALT                        0
Alkaline Phos              0
HbA1c                      0
dtype: int64

### KNN

In [108]:
from sklearn import neighbors
from sklearn.impute import KNNImputer

In [109]:
t_start = time.time()
KNN_imputer = KNNImputer(n_neighbors = 15)
data_KNN10 = KNN_imputer.fit_transform(dt10.copy())
t_end = time.time()
t_KNN10= (t_end- t_start)

In [110]:
data_KNN10 = pd.DataFrame(data_KNN10)

In [111]:
data_KNN10 = data_KNN10.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})
data_KNN10

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,57.0,47.4,153.0,20.20,114.0,75.0,69.0,21.286667,12.500000,...,1.0,5.000000,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.100000
1,2.0,24.0,53.2,157.0,21.60,103.0,69.0,78.0,21.533333,11.873333,...,0.8,5.200000,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.400000
2,2.0,62.0,54.4,165.0,20.00,95.0,75.0,83.0,21.600000,15.200000,...,0.7,4.100000,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.600000
3,2.0,35.0,56.3,161.0,21.70,95.0,57.0,72.0,22.500000,15.400000,...,0.6,5.000000,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.100000
4,1.0,46.0,72.3,170.0,21.72,133.0,77.0,107.0,21.826667,19.000000,...,0.9,6.900000,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1.0,60.0,72.0,170.0,24.90,131.0,74.0,69.0,30.500000,13.800000,...,0.9,6.900000,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.300000
149,1.0,58.0,62.0,168.0,22.00,103.0,76.0,72.0,27.800000,11.900000,...,1.0,6.800000,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.800000
150,2.0,42.0,55.5,165.0,20.40,102.0,52.0,77.0,22.900000,14.100000,...,0.8,4.700000,212.0,68.0,63.0,135.0,11.0,16.0,33.0,5.206667
151,2.0,61.0,61.8,159.0,24.40,117.0,60.0,72.0,23.400000,19.200000,...,0.8,5.100000,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.533333


In [112]:
mse_KNN10 = mean_squared_error(data1,data_KNN10)
mse_KNN10 

10.106348141556339

### MICE

In [113]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn import svm

#### LinearRegression model

In [ ]:
t_start = time.time()
lr = LinearRegression()
imp = IterativeImputer(estimator= lr , max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean')
data_LR10 = imp.fit_transform(dt10.copy())
t_end = time.time()
t_LR10 = (t_end- t_start)

In [115]:
data_LR10 = pd.DataFrame(data_LR10)

In [116]:
data_LR10 = data_LR10.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})
data_LR10

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,57.0,47.4,153.0,20.200000,114.0,75.0,69.0,18.280032,12.500000,...,1.0,5.000000,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.100000
1,2.0,24.0,53.2,157.0,21.600000,103.0,69.0,78.0,20.652962,15.628888,...,0.8,5.200000,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.400000
2,2.0,62.0,54.4,165.0,20.000000,95.0,75.0,83.0,21.600000,15.200000,...,0.7,4.100000,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.600000
3,2.0,35.0,56.3,161.0,21.700000,95.0,57.0,72.0,22.500000,15.400000,...,0.6,5.000000,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.100000
4,1.0,46.0,72.3,170.0,24.847401,133.0,77.0,107.0,32.416688,19.000000,...,0.9,6.900000,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1.0,60.0,72.0,170.0,24.900000,131.0,74.0,69.0,30.500000,13.800000,...,0.9,6.900000,182.0,179.0,39.0,107.0,36.0,31.0,79.0,5.914207
149,1.0,58.0,62.0,168.0,22.000000,103.0,76.0,72.0,27.800000,11.900000,...,1.0,6.800000,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.800000
150,2.0,42.0,55.5,165.0,20.400000,102.0,52.0,77.0,22.900000,14.100000,...,0.8,4.700000,212.0,68.0,63.0,135.0,11.0,16.0,33.0,5.184096
151,2.0,61.0,61.8,159.0,24.400000,117.0,60.0,72.0,23.400000,19.200000,...,0.8,5.100000,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.569468


In [118]:
mse_LR10 = mean_squared_error(data1,data_LR10)
mse_LR10 

2.246907836639976

#### Support Vector Machine model

In [119]:
t_start = time.time()
regr = svm.SVR()
svm_svr = IterativeImputer(estimator= regr, max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean')
data_SVM10 = svm_svr.fit_transform(dt10.copy())
t_end = time.time()
t_SVM10 = (t_end- t_start)

[IterativeImputer] Completing matrix with shape (153, 26)
[IterativeImputer] Ending imputation round 1/10, elapsed time 0.17
[IterativeImputer] Change: 17.977680914620983, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 2/10, elapsed time 0.31
[IterativeImputer] Change: 0.605286166198507, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 3/10, elapsed time 0.47
[IterativeImputer] Change: 0.004405024558577608, scaled tolerance: 0.274 
[IterativeImputer] Early stopping criterion reached.


In [120]:
data_SVM10 = pd.DataFrame(data_SVM10)

In [121]:
data_SVM10 = data_SVM10.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})
data_SVM10

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,57.0,47.4,153.0,20.20000,114.0,75.0,69.0,21.062953,12.500000,...,1.0,5.00000,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.100000
1,2.0,24.0,53.2,157.0,21.60000,103.0,69.0,78.0,21.479244,13.237078,...,0.8,5.20000,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.400000
2,2.0,62.0,54.4,165.0,20.00000,95.0,75.0,83.0,21.600000,15.200000,...,0.7,4.10000,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.600000
3,2.0,35.0,56.3,161.0,21.70000,95.0,57.0,72.0,22.500000,15.400000,...,0.6,5.00000,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.100000
4,1.0,46.0,72.3,170.0,21.04572,133.0,77.0,107.0,22.361224,19.000000,...,0.9,6.90000,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1.0,60.0,72.0,170.0,24.90000,131.0,74.0,69.0,30.500000,13.800000,...,0.9,6.90000,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.015541
149,1.0,58.0,62.0,168.0,22.00000,103.0,76.0,72.0,27.800000,11.900000,...,1.0,6.80000,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.800000
150,2.0,42.0,55.5,165.0,20.40000,102.0,52.0,77.0,22.900000,14.100000,...,0.8,4.70000,212.0,68.0,63.0,135.0,11.0,16.0,33.0,5.426698
151,2.0,61.0,61.8,159.0,24.40000,117.0,60.0,72.0,23.400000,19.200000,...,0.8,5.10000,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.757843


In [122]:
mse_SVM10 = mean_squared_error(data1,data_SVM10)
mse_SVM10 

15.029067863192903

### EM

In [123]:
!pip install impyute
import impyute as impy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [124]:
t_start = time.time()
data_EM10 = impy.em(dt10.values.copy())
t_end = time.time()
t_EM10 = (t_end- t_start)

In [125]:
data_EM10 = pd.DataFrame(data_EM10)

In [126]:
data_EM10 = data_EM10.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})
data_EM10

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,57.0,47.4,153.0,20.200000,114.0,75.0,69.0,18.303940,12.500000,...,1.0,5.000000,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.100000
1,2.0,24.0,53.2,157.0,21.600000,103.0,69.0,78.0,19.770949,11.074791,...,0.8,5.200000,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.400000
2,2.0,62.0,54.4,165.0,20.000000,95.0,75.0,83.0,21.600000,15.200000,...,0.7,4.100000,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.600000
3,2.0,35.0,56.3,161.0,21.700000,95.0,57.0,72.0,22.500000,15.400000,...,0.6,5.000000,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.100000
4,1.0,46.0,72.3,170.0,21.532832,133.0,77.0,107.0,20.308641,19.000000,...,0.9,6.900000,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1.0,60.0,72.0,170.0,24.900000,131.0,74.0,69.0,30.500000,13.800000,...,0.9,6.900000,182.0,179.0,39.0,107.0,36.0,31.0,79.0,2.552141
149,1.0,58.0,62.0,168.0,22.000000,103.0,76.0,72.0,27.800000,11.900000,...,1.0,6.800000,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.800000
150,2.0,42.0,55.5,165.0,20.400000,102.0,52.0,77.0,22.900000,14.100000,...,0.8,4.700000,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.644677
151,2.0,61.0,61.8,159.0,24.400000,117.0,60.0,72.0,23.400000,19.200000,...,0.8,5.100000,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.093866


In [127]:
mse_EM10 = mean_squared_error(data1,data_EM10)
mse_EM10 

25.21181422704657

# 20 %

## แทนค่า NaN

In [ ]:
random_column = np.random.choice(list1,int((153*17)*(20/100)),p=list_mis)
random_column = list(random_column)
random_column

In [129]:
weight = []
height = []
bmi = []
systolic = []
diastolic = []
heartrate = []
smm = []
fatmass = []
bodyfat = []
wait = []
abdominal = []
visceral = []
glucose = []
uric_acid = []
ALT = []
Alkaline = []
HbA1c = []

In [130]:
for i in random_column:
  if i == 'Weight':
    weight.append(i)  
  elif i == 'Height':
    height.append(i)
  elif i == 'BMI':
    bmi.append(i)
  elif i == 'Systolic':
    systolic.append(i)
  elif i == 'Diastolic':
    diastolic.append(i)
  elif i == 'HeartRate':
    heartrate.append(i)
  elif i == 'SMM':
    smm.append(i)
  elif i == 'FatMass':
    fatmass.append(i)
  elif i == 'Bodyfat':
    bodyfat.append(i)
  elif i == 'waisttohip':
    wait.append(i)
  elif i == 'abdominal':
    abdominal.append(i)
  elif i == 'visceral':
    visceral.append(i)
  elif i == '001_Glucose':
    glucose.append(i)
  elif i == 'UricAcid':
    uric_acid.append(i)
  elif i == '012_ALT':
    ALT.append(i)
  elif i == 'Alkaline_Phos':
    Alkaline.append(i)
  elif i == '071_HbA1c':
    HbA1c.append(i)

In [131]:
list2 = [len(weight),len(height),len(bmi),len(systolic),len(diastolic),len(heartrate),len(smm),
         len(fatmass),len(bodyfat),len(wait),len(abdominal),len(visceral),len(glucose),len(uric_acid),
         len(ALT),len(Alkaline),len(HbA1c)]
list3 = []
list4 = []

for i in range(153):
  m = 1/153
  list3.append(i)
  list4.append(m)

In [132]:
list2

[2, 2, 1, 1, 3, 20, 80, 86, 72, 51, 72, 58, 1, 35, 0, 4, 32]

In [133]:
list1

['Weight',
 'Height',
 'BMI',
 'Systolic',
 'Diastolic',
 'HeartRate',
 'SMM',
 'FatMass',
 'Bodyfat',
 'waisttohip',
 'abdominal',
 'visceral',
 '001_Glucose',
 'UricAcid',
 '012_ALT',
 'Alkaline_Phos',
 '071_HbA1c']

In [134]:
dt20 = data1.copy()
dt20

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,FatMass,...,003_Creatinine,UricAcid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,Alkaline_Phos,071_HbA1c
0,2,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2,62.0,54.4,165.0,20.0,95.0,75.0,83.0,21.6,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.5,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1,46.0,72.3,170.0,25.0,133.0,77.0,107.0,29.2,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,72.0,170.0,24.9,131.0,74.0,69.0,30.5,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,27.8,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.9,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2,61.0,61.8,159.0,24.4,117.0,60.0,72.0,23.4,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


In [135]:
for i,j in zip(list1,list2):
  dt20[i].ravel()[np.random.choice(list3,j,p= list4, replace=False)] = np.nan

In [136]:
dt20.isna().sum()

Sex                  0
AgeSYear             0
Weight               2
Height               2
BMI                  1
Systolic             1
Diastolic            3
HeartRate           20
SMM                 80
FatMass             86
Bodyfat             72
waisttohip          51
abdominal           72
visceral            58
001_Glucose          1
002_BUN              0
003_Creatinine       0
UricAcid            35
005_Cholesterol      0
006_Triglyceride     0
008_HDL-C            0
009_LDL              0
010_AST              0
012_ALT              0
Alkaline_Phos        4
071_HbA1c           32
dtype: int64

In [137]:
dt20

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,FatMass,...,003_Creatinine,UricAcid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,Alkaline_Phos,071_HbA1c
0,2,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.3,NaN,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,NaN,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2,62.0,54.4,165.0,20.0,95.0,75.0,NaN,21.6,NaN,...,0.7,NaN,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.5,NaN,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1,46.0,72.3,170.0,25.0,133.0,77.0,107.0,29.2,NaN,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,72.0,170.0,24.9,131.0,74.0,69.0,30.5,NaN,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,NaN,6.5
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,NaN,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,NaN,NaN,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2,61.0,61.8,159.0,24.4,117.0,60.0,72.0,23.4,NaN,...,0.8,NaN,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


## Imputation

### MEAN

In [138]:
from sklearn.impute import SimpleImputer

In [139]:
t_start = time.time()
mean_imputer = SimpleImputer(strategy='mean')
data_MEAN20 = mean_imputer.fit_transform(dt20.copy())
t_end = time.time()
t_MEAN20 = (t_end- t_start)

In [140]:
data_MEAN20 = pd.DataFrame(data_MEAN20)
data_MEAN20 = data_MEAN20.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})
data_MEAN20

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,57.0,47.4,153.0,20.2,114.0,75.0,69.000000,19.300000,14.546269,...,1.0,5.000000,176.0,51.0,72.0,94.0,16.0,16.0,53.90604,5.100000
1,2.0,24.0,53.2,157.0,21.6,103.0,69.0,78.000000,21.300000,14.600000,...,0.8,5.200000,159.0,45.0,52.0,98.0,13.0,17.0,50.00000,5.400000
2,2.0,62.0,54.4,165.0,20.0,95.0,75.0,75.849624,21.600000,14.546269,...,0.7,4.874576,230.0,94.0,61.0,151.0,18.0,31.0,76.00000,5.600000
3,2.0,35.0,56.3,161.0,21.7,95.0,57.0,72.000000,22.500000,14.546269,...,0.6,5.000000,198.0,47.0,68.0,120.0,16.0,20.0,35.00000,5.100000
4,1.0,46.0,72.3,170.0,25.0,133.0,77.0,107.000000,29.200000,14.546269,...,0.9,6.900000,194.0,63.0,60.0,122.0,15.0,19.0,69.00000,5.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1.0,60.0,72.0,170.0,24.9,131.0,74.0,69.000000,30.500000,14.546269,...,0.9,6.900000,182.0,179.0,39.0,107.0,36.0,31.0,53.90604,6.500000
149,1.0,58.0,62.0,168.0,22.0,103.0,76.0,72.000000,22.976712,11.900000,...,1.0,6.800000,191.0,62.0,47.0,132.0,16.0,33.0,42.00000,6.800000
150,2.0,42.0,55.5,165.0,20.4,102.0,52.0,77.000000,22.976712,14.546269,...,0.8,4.700000,212.0,68.0,63.0,135.0,11.0,16.0,33.00000,4.800000
151,2.0,61.0,61.8,159.0,24.4,117.0,60.0,72.000000,23.400000,14.546269,...,0.8,4.874576,235.0,66.0,56.0,166.0,15.0,23.0,44.00000,5.400000


In [141]:
mse_MEAN20 = mean_squared_error(data1,data_MEAN20)
mse_MEAN20

21.64090128148532

### MISS Forest

In [142]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install missingpy
import missingpy
from missingpy import MissForest
from sklearn.metrics import mean_squared_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
t_start = time.time()
MF = MissForest()
data_MF20 = MF.fit_transform(dt20.copy())
t_end = time.time()
t_MF20 = (t_end- t_start)

In [144]:
data_MF20 = pd.DataFrame(data_MF20)

In [145]:
data_MF20 = data_MF20.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})
data_MF20

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,57.0,47.4,153.0,20.2,114.0,75.0,69.00,19.300,12.436,...,1.0,5.000,176.0,51.0,72.0,94.0,16.0,16.0,48.06,5.100
1,2.0,24.0,53.2,157.0,21.6,103.0,69.0,78.00,21.300,14.600,...,0.8,5.200,159.0,45.0,52.0,98.0,13.0,17.0,50.00,5.400
2,2.0,62.0,54.4,165.0,20.0,95.0,75.0,71.42,21.600,12.764,...,0.7,4.257,230.0,94.0,61.0,151.0,18.0,31.0,76.00,5.600
3,2.0,35.0,56.3,161.0,21.7,95.0,57.0,72.00,22.500,17.239,...,0.6,5.000,198.0,47.0,68.0,120.0,16.0,20.0,35.00,5.100
4,1.0,46.0,72.3,170.0,25.0,133.0,77.0,107.00,29.200,13.617,...,0.9,6.900,194.0,63.0,60.0,122.0,15.0,19.0,69.00,5.200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1.0,60.0,72.0,170.0,24.9,131.0,74.0,69.00,30.500,13.098,...,0.9,6.900,182.0,179.0,39.0,107.0,36.0,31.0,67.91,6.500
149,1.0,58.0,62.0,168.0,22.0,103.0,76.0,72.00,29.113,11.900,...,1.0,6.800,191.0,62.0,47.0,132.0,16.0,33.0,42.00,6.800
150,2.0,42.0,55.5,165.0,20.4,102.0,52.0,77.00,22.614,13.375,...,0.8,4.700,212.0,68.0,63.0,135.0,11.0,16.0,33.00,4.800
151,2.0,61.0,61.8,159.0,24.4,117.0,60.0,72.00,23.400,18.748,...,0.8,4.513,235.0,66.0,56.0,166.0,15.0,23.0,44.00,5.400


In [146]:
mse_MF20 = mean_squared_error(data1,data_MF20)
mse_MF20

6.124488152144293

In [147]:
data_MF20.isna().sum()

Sex                        0
AgeSYear                   0
Weight                     0
Height                     0
BMI                        0
Systolic                   0
Diastolic                  0
HeartRate                  0
SMM                        0
Fat Mass                   0
% Body fat                 0
waist to hip radio         0
abdominal circumference    0
visceral fat area          0
Glucose                    0
BUN                        0
Creatinine                 0
Uric Acid                  0
Cholesterol                0
Triglyceride               0
HDL-C                      0
LDL                        0
AST                        0
ALT                        0
Alkaline Phos              0
HbA1c                      0
dtype: int64

### KNN

In [148]:
from sklearn import neighbors
from sklearn.impute import KNNImputer

In [149]:
t_start = time.time()
KNN_imputer = KNNImputer(n_neighbors = 15)
data_KNN20 = KNN_imputer.fit_transform(dt20.copy())
t_end = time.time()
t_KNN20= (t_end- t_start)

In [150]:
data_KNN20 = pd.DataFrame(data_KNN20)

In [151]:
data_KNN20 = data_KNN20.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})
data_KNN20

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,57.0,47.4,153.0,20.2,114.0,75.0,69.000000,19.300000,13.700000,...,1.0,5.000000,176.0,51.0,72.0,94.0,16.0,16.0,46.066667,5.10
1,2.0,24.0,53.2,157.0,21.6,103.0,69.0,78.000000,21.300000,14.600000,...,0.8,5.200000,159.0,45.0,52.0,98.0,13.0,17.0,50.000000,5.40
2,2.0,62.0,54.4,165.0,20.0,95.0,75.0,73.733333,21.600000,16.380000,...,0.7,5.066667,230.0,94.0,61.0,151.0,18.0,31.0,76.000000,5.60
3,2.0,35.0,56.3,161.0,21.7,95.0,57.0,72.000000,22.500000,13.760000,...,0.6,5.000000,198.0,47.0,68.0,120.0,16.0,20.0,35.000000,5.10
4,1.0,46.0,72.3,170.0,25.0,133.0,77.0,107.000000,29.200000,13.773333,...,0.9,6.900000,194.0,63.0,60.0,122.0,15.0,19.0,69.000000,5.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1.0,60.0,72.0,170.0,24.9,131.0,74.0,69.000000,30.500000,16.973333,...,0.9,6.900000,182.0,179.0,39.0,107.0,36.0,31.0,63.466667,6.50
149,1.0,58.0,62.0,168.0,22.0,103.0,76.0,72.000000,23.826667,11.900000,...,1.0,6.800000,191.0,62.0,47.0,132.0,16.0,33.0,42.000000,6.80
150,2.0,42.0,55.5,165.0,20.4,102.0,52.0,77.000000,22.340000,14.466667,...,0.8,4.700000,212.0,68.0,63.0,135.0,11.0,16.0,33.000000,4.80
151,2.0,61.0,61.8,159.0,24.4,117.0,60.0,72.000000,23.400000,14.226667,...,0.8,4.820000,235.0,66.0,56.0,166.0,15.0,23.0,44.000000,5.40


In [152]:
mse_KNN20 = mean_squared_error(data1,data_KNN20)
mse_KNN20 

16.685132145131558

### MICE

In [153]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn import svm

#### LinearRegression model

In [154]:
t_start = time.time()
lr = LinearRegression()
imp = IterativeImputer(estimator= lr , max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean')
data_LR20 = imp.fit_transform(dt20.copy())
t_end = time.time()
t_LR20 = (t_end- t_start)

[IterativeImputer] Completing matrix with shape (153, 26)
[IterativeImputer] Ending imputation round 1/10, elapsed time 0.07
[IterativeImputer] Change: 94.7299073012677, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 2/10, elapsed time 0.16
[IterativeImputer] Change: 20.99441769133, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 3/10, elapsed time 0.22
[IterativeImputer] Change: 17.159352395805342, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 4/10, elapsed time 0.28
[IterativeImputer] Change: 2.257154586410252, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 5/10, elapsed time 0.33
[IterativeImputer] Change: 0.3797992698109738, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 6/10, elapsed time 0.40
[IterativeImputer] Change: 0.092608279104752, scaled tolerance: 0.274 
[IterativeImputer] Early stopping criterion reached.


In [155]:
data_LR20 = pd.DataFrame(data_LR20)

In [156]:
data_LR20 = data_LR20.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})
data_LR20

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,57.0,47.4,153.0,20.2,114.0,75.0,69.000000,19.300000,9.999771,...,1.0,5.000000,176.0,51.0,72.0,94.0,16.0,16.0,46.409492,5.100000
1,2.0,24.0,53.2,157.0,21.6,103.0,69.0,78.000000,21.300000,14.600000,...,0.8,5.200000,159.0,45.0,52.0,98.0,13.0,17.0,50.000000,5.400000
2,2.0,62.0,54.4,165.0,20.0,95.0,75.0,69.609028,21.600000,16.270425,...,0.7,4.601815,230.0,94.0,61.0,151.0,18.0,31.0,76.000000,5.600000
3,2.0,35.0,56.3,161.0,21.7,95.0,57.0,72.000000,22.500000,15.340817,...,0.6,5.000000,198.0,47.0,68.0,120.0,16.0,20.0,35.000000,5.100000
4,1.0,46.0,72.3,170.0,25.0,133.0,77.0,107.000000,29.200000,15.592890,...,0.9,6.900000,194.0,63.0,60.0,122.0,15.0,19.0,69.000000,5.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1.0,60.0,72.0,170.0,24.9,131.0,74.0,69.000000,30.500000,13.093749,...,0.9,6.900000,182.0,179.0,39.0,107.0,36.0,31.0,68.176638,6.500000
149,1.0,58.0,62.0,168.0,22.0,103.0,76.0,72.000000,27.740746,11.900000,...,1.0,6.800000,191.0,62.0,47.0,132.0,16.0,33.0,42.000000,6.800000
150,2.0,42.0,55.5,165.0,20.4,102.0,52.0,77.000000,22.937340,15.145667,...,0.8,4.700000,212.0,68.0,63.0,135.0,11.0,16.0,33.000000,4.800000
151,2.0,61.0,61.8,159.0,24.4,117.0,60.0,72.000000,23.400000,18.740984,...,0.8,4.245715,235.0,66.0,56.0,166.0,15.0,23.0,44.000000,5.400000


In [157]:
mse_LR20 = mean_squared_error(data1,data_LR20)
mse_LR20 

7.414224539244076

#### Support Vector Machine model

In [158]:
t_start = time.time()
regr = svm.SVR()
svm_svr = IterativeImputer(estimator= regr, max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean')
data_SVM20 = svm_svr.fit_transform(dt20.copy())
t_end = time.time()
t_SVM20 = (t_end- t_start)

[IterativeImputer] Completing matrix with shape (153, 26)
[IterativeImputer] Ending imputation round 1/10, elapsed time 0.15
[IterativeImputer] Change: 19.129476588260353, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 2/10, elapsed time 0.28
[IterativeImputer] Change: 0.5681045268965048, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 3/10, elapsed time 0.40
[IterativeImputer] Change: 0.003400293616515171, scaled tolerance: 0.274 
[IterativeImputer] Early stopping criterion reached.


In [159]:
data_SVM20 = pd.DataFrame(data_SVM20)

In [160]:
data_SVM20 = data_SVM20.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})
data_SVM20

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,57.0,47.4,153.0,20.2,114.0,75.0,69.00000,19.30000,13.444932,...,1.0,5.000000,176.0,51.0,72.0,94.0,16.0,16.0,51.177889,5.100000
1,2.0,24.0,53.2,157.0,21.6,103.0,69.0,78.00000,21.30000,14.600000,...,0.8,5.200000,159.0,45.0,52.0,98.0,13.0,17.0,50.000000,5.400000
2,2.0,62.0,54.4,165.0,20.0,95.0,75.0,73.14061,21.60000,14.500507,...,0.7,4.854233,230.0,94.0,61.0,151.0,18.0,31.0,76.000000,5.600000
3,2.0,35.0,56.3,161.0,21.7,95.0,57.0,72.00000,22.50000,13.682220,...,0.6,5.000000,198.0,47.0,68.0,120.0,16.0,20.0,35.000000,5.100000
4,1.0,46.0,72.3,170.0,25.0,133.0,77.0,107.00000,29.20000,13.832798,...,0.9,6.900000,194.0,63.0,60.0,122.0,15.0,19.0,69.000000,5.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1.0,60.0,72.0,170.0,24.9,131.0,74.0,69.00000,30.50000,14.357452,...,0.9,6.900000,182.0,179.0,39.0,107.0,36.0,31.0,54.132947,6.500000
149,1.0,58.0,62.0,168.0,22.0,103.0,76.0,72.00000,22.31815,11.900000,...,1.0,6.800000,191.0,62.0,47.0,132.0,16.0,33.0,42.000000,6.800000
150,2.0,42.0,55.5,165.0,20.4,102.0,52.0,77.00000,21.78508,14.008426,...,0.8,4.700000,212.0,68.0,63.0,135.0,11.0,16.0,33.000000,4.800000
151,2.0,61.0,61.8,159.0,24.4,117.0,60.0,72.00000,23.40000,14.512449,...,0.8,4.921654,235.0,66.0,56.0,166.0,15.0,23.0,44.000000,5.400000


In [161]:
mse_SVM20 = mean_squared_error(data1,data_SVM20)
mse_SVM20 

23.465243332508653

### EM

In [162]:
!pip install impyute
import impyute as impy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [163]:
t_start = time.time()
data_EM20 = impy.em(dt20.values.copy())
t_end = time.time()
t_EM20 = (t_end- t_start)

In [164]:
data_EM20 = pd.DataFrame(data_EM20)

In [165]:
data_EM20 = data_EM20.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})
data_EM20

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,57.0,47.4,153.0,20.2,114.0,75.0,69.000000,19.300000,1.998288,...,1.0,5.000000,176.0,51.0,72.0,94.0,16.0,16.0,51.897714,5.100000
1,2.0,24.0,53.2,157.0,21.6,103.0,69.0,78.000000,21.300000,14.600000,...,0.8,5.200000,159.0,45.0,52.0,98.0,13.0,17.0,50.000000,5.400000
2,2.0,62.0,54.4,165.0,20.0,95.0,75.0,78.376708,21.600000,18.368947,...,0.7,4.942258,230.0,94.0,61.0,151.0,18.0,31.0,76.000000,5.600000
3,2.0,35.0,56.3,161.0,21.7,95.0,57.0,72.000000,22.500000,13.150191,...,0.6,5.000000,198.0,47.0,68.0,120.0,16.0,20.0,35.000000,5.100000
4,1.0,46.0,72.3,170.0,25.0,133.0,77.0,107.000000,29.200000,14.321997,...,0.9,6.900000,194.0,63.0,60.0,122.0,15.0,19.0,69.000000,5.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1.0,60.0,72.0,170.0,24.9,131.0,74.0,69.000000,30.500000,10.194299,...,0.9,6.900000,182.0,179.0,39.0,107.0,36.0,31.0,56.584971,6.500000
149,1.0,58.0,62.0,168.0,22.0,103.0,76.0,72.000000,15.185246,11.900000,...,1.0,6.800000,191.0,62.0,47.0,132.0,16.0,33.0,42.000000,6.800000
150,2.0,42.0,55.5,165.0,20.4,102.0,52.0,77.000000,18.096757,13.825371,...,0.8,4.700000,212.0,68.0,63.0,135.0,11.0,16.0,33.000000,4.800000
151,2.0,61.0,61.8,159.0,24.4,117.0,60.0,72.000000,23.400000,10.727308,...,0.8,3.525670,235.0,66.0,56.0,166.0,15.0,23.0,44.000000,5.400000


In [166]:
mse_EM20 = mean_squared_error(data1,data_EM20)
mse_EM20 

40.297897544147226

# 30 %

## แทนค่า NaN

In [ ]:
random_column = np.random.choice(list1,int((153*17)*(30/100)),p=list_mis)
random_column = list(random_column)
random_column

In [168]:
weight = []
height = []
bmi = []
systolic = []
diastolic = []
heartrate = []
smm = []
fatmass = []
bodyfat = []
wait = []
abdominal = []
visceral = []
glucose = []
uric_acid = []
ALT = []
Alkaline = []
HbA1c = []

In [169]:
for i in random_column:
  if i == 'Weight':
    weight.append(i)  
  elif i == 'Height':
    height.append(i)
  elif i == 'BMI':
    bmi.append(i)
  elif i == 'Systolic':
    systolic.append(i)
  elif i == 'Diastolic':
    diastolic.append(i)
  elif i == 'HeartRate':
    heartrate.append(i)
  elif i == 'SMM':
    smm.append(i)
  elif i == 'FatMass':
    fatmass.append(i)
  elif i == 'Bodyfat':
    bodyfat.append(i)
  elif i == 'waisttohip':
    wait.append(i)
  elif i == 'abdominal':
    abdominal.append(i)
  elif i == 'visceral':
    visceral.append(i)
  elif i == '001_Glucose':
    glucose.append(i)
  elif i == 'UricAcid':
    uric_acid.append(i)
  elif i == '012_ALT':
    ALT.append(i)
  elif i == 'Alkaline_Phos':
    Alkaline.append(i)
  elif i == '071_HbA1c':
    HbA1c.append(i)

In [170]:
list2 = [len(weight),len(height),len(bmi),len(systolic),len(diastolic),len(heartrate),len(smm),
         len(fatmass),len(bodyfat),len(wait),len(abdominal),len(visceral),len(glucose),len(uric_acid),
         len(ALT),len(Alkaline),len(HbA1c)]
list3 = []
list4 = []

for i in range(153):
  m = 1/153
  list3.append(i)
  list4.append(m)

In [171]:
list2

[3, 5, 6, 4, 2, 15, 104, 109, 102, 101, 94, 108, 3, 40, 2, 12, 70]

In [172]:
list1

['Weight',
 'Height',
 'BMI',
 'Systolic',
 'Diastolic',
 'HeartRate',
 'SMM',
 'FatMass',
 'Bodyfat',
 'waisttohip',
 'abdominal',
 'visceral',
 '001_Glucose',
 'UricAcid',
 '012_ALT',
 'Alkaline_Phos',
 '071_HbA1c']

In [173]:
dt30 = data1.copy()
dt30

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,FatMass,...,003_Creatinine,UricAcid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,Alkaline_Phos,071_HbA1c
0,2,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2,62.0,54.4,165.0,20.0,95.0,75.0,83.0,21.6,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.5,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1,46.0,72.3,170.0,25.0,133.0,77.0,107.0,29.2,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,72.0,170.0,24.9,131.0,74.0,69.0,30.5,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,27.8,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.9,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2,61.0,61.8,159.0,24.4,117.0,60.0,72.0,23.4,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


In [174]:
for i,j in zip(list1,list2):
  dt30[i].ravel()[np.random.choice(list3,j,p= list4, replace=False)] = np.nan

In [175]:
dt30.isna().sum()

Sex                   0
AgeSYear              0
Weight                3
Height                5
BMI                   6
Systolic              4
Diastolic             2
HeartRate            15
SMM                 104
FatMass             109
Bodyfat             102
waisttohip          101
abdominal            94
visceral            108
001_Glucose           3
002_BUN               0
003_Creatinine        0
UricAcid             40
005_Cholesterol       0
006_Triglyceride      0
008_HDL-C             0
009_LDL               0
010_AST               0
012_ALT               2
Alkaline_Phos        12
071_HbA1c            70
dtype: int64

In [176]:
dt30

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,FatMass,...,003_Creatinine,UricAcid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,Alkaline_Phos,071_HbA1c
0,2,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.3,NaN,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,NaN,NaN,NaN,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2,62.0,54.4,165.0,20.0,95.0,75.0,83.0,21.6,NaN,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.5,NaN,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,NaN
4,1,46.0,72.3,170.0,25.0,133.0,77.0,NaN,NaN,NaN,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,72.0,170.0,24.9,131.0,74.0,69.0,NaN,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,NaN
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,NaN,NaN,...,1.0,NaN,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,NaN,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,NaN
151,2,61.0,61.8,159.0,24.4,117.0,60.0,72.0,23.4,NaN,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,NaN,5.4


## Imputation

### MEAN

In [177]:
from sklearn.impute import SimpleImputer

In [178]:
t_start = time.time()
mean_imputer = SimpleImputer(strategy='mean')
data_MEAN30 = mean_imputer.fit_transform(dt30.copy())
t_end = time.time()
t_MEAN30 = (t_end- t_start)

In [179]:
data_MEAN30 = pd.DataFrame(data_MEAN30)
data_MEAN30 = data_MEAN10.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})
data_MEAN30

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,57.0,47.4,153.0,20.200000,114.0,75.0,69.0,23.042373,12.500000,...,1.0,5.00000,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.100000
1,2.0,24.0,53.2,157.0,21.600000,103.0,69.0,78.0,23.042373,14.726667,...,0.8,5.20000,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.400000
2,2.0,62.0,54.4,165.0,20.000000,95.0,75.0,83.0,21.600000,15.200000,...,0.7,4.10000,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.600000
3,2.0,35.0,56.3,161.0,21.700000,95.0,57.0,72.0,22.500000,15.400000,...,0.6,5.00000,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.100000
4,1.0,46.0,72.3,170.0,21.422517,133.0,77.0,107.0,23.042373,19.000000,...,0.9,6.90000,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1.0,60.0,72.0,170.0,24.900000,131.0,74.0,69.0,30.500000,13.800000,...,0.9,6.90000,182.0,179.0,39.0,107.0,36.0,31.0,79.0,5.563359
149,1.0,58.0,62.0,168.0,22.000000,103.0,76.0,72.0,27.800000,11.900000,...,1.0,6.80000,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.800000
150,2.0,42.0,55.5,165.0,20.400000,102.0,52.0,77.0,22.900000,14.100000,...,0.8,4.70000,212.0,68.0,63.0,135.0,11.0,16.0,33.0,5.563359
151,2.0,61.0,61.8,159.0,24.400000,117.0,60.0,72.0,23.400000,19.200000,...,0.8,5.10000,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.563359


In [180]:
mse_MEAN30 = mean_squared_error(data1,data_MEAN30)
mse_MEAN30

14.794385185765863

### MISS Forest

In [181]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install missingpy
import missingpy
from missingpy import MissForest
from sklearn.metrics import mean_squared_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
t_start = time.time()
MF = MissForest()
data_MF30 = MF.fit_transform(dt30.copy())
t_end = time.time()
t_MF30= (t_end- t_start)

In [183]:
data_MF30 = pd.DataFrame(data_MF30)

In [184]:
data_MF30 = data_MF30.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})
data_MF30

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,57.0,47.4,153.0,20.2,114.0,75.0,69.00,19.300,13.035,...,1.0,5.000,176.0,51.0,72.0,94.0,16.0,16.0,62.00,5.100
1,2.0,24.0,53.2,157.0,21.6,103.0,69.0,77.12,20.509,13.708,...,0.8,5.200,159.0,45.0,52.0,98.0,13.0,17.0,50.00,5.400
2,2.0,62.0,54.4,165.0,20.0,95.0,75.0,83.00,21.600,10.958,...,0.7,4.100,230.0,94.0,61.0,151.0,18.0,31.0,76.00,5.600
3,2.0,35.0,56.3,161.0,21.7,95.0,57.0,72.00,22.500,18.381,...,0.6,5.000,198.0,47.0,68.0,120.0,16.0,20.0,35.00,4.806
4,1.0,46.0,72.3,170.0,25.0,133.0,77.0,73.47,28.435,13.629,...,0.9,6.900,194.0,63.0,60.0,122.0,15.0,19.0,69.00,5.555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1.0,60.0,72.0,170.0,24.9,131.0,74.0,69.00,28.414,13.800,...,0.9,6.900,182.0,179.0,39.0,107.0,36.0,31.0,79.00,6.768
149,1.0,58.0,62.0,168.0,22.0,103.0,76.0,72.00,27.743,13.327,...,1.0,6.133,191.0,62.0,47.0,132.0,16.0,33.0,42.00,6.800
150,2.0,42.0,55.5,165.0,20.4,102.0,52.0,77.00,22.094,14.100,...,0.8,4.700,212.0,68.0,63.0,135.0,11.0,16.0,33.00,5.107
151,2.0,61.0,61.8,159.0,24.4,117.0,60.0,72.00,23.400,17.137,...,0.8,5.100,235.0,66.0,56.0,166.0,15.0,23.0,57.55,5.400


In [185]:
mse_MF30 = mean_squared_error(data1,data_MF30)
mse_MF30

13.375977123891401

In [186]:
data_MF30.isna().sum()

Sex                        0
AgeSYear                   0
Weight                     0
Height                     0
BMI                        0
Systolic                   0
Diastolic                  0
HeartRate                  0
SMM                        0
Fat Mass                   0
% Body fat                 0
waist to hip radio         0
abdominal circumference    0
visceral fat area          0
Glucose                    0
BUN                        0
Creatinine                 0
Uric Acid                  0
Cholesterol                0
Triglyceride               0
HDL-C                      0
LDL                        0
AST                        0
ALT                        0
Alkaline Phos              0
HbA1c                      0
dtype: int64

### KNN

In [187]:
from sklearn import neighbors
from sklearn.impute import KNNImputer

In [188]:
t_start = time.time()
KNN_imputer = KNNImputer(n_neighbors = 15)
data_KNN30 = KNN_imputer.fit_transform(dt30.copy())
t_end = time.time()
t_KNN30= (t_end- t_start)

In [189]:
data_KNN30 = pd.DataFrame(data_KNN30)

In [190]:
data_KNN30 = data_KNN30.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})
data_KNN30

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.300000,13.553333,...,1.0,5.000000,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.100000
1,2.0,24.0,53.2,157.0,21.6,103.0,69.0,74.8,21.580000,14.646667,...,0.8,5.200000,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.400000
2,2.0,62.0,54.4,165.0,20.0,95.0,75.0,83.0,21.600000,16.786667,...,0.7,4.100000,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.600000
3,2.0,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.500000,16.653333,...,0.6,5.000000,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.006667
4,1.0,46.0,72.3,170.0,25.0,133.0,77.0,73.2,22.220000,14.753333,...,0.9,6.900000,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.133333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1.0,60.0,72.0,170.0,24.9,131.0,74.0,69.0,21.853333,13.800000,...,0.9,6.900000,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.520000
149,1.0,58.0,62.0,168.0,22.0,103.0,76.0,72.0,22.300000,15.380000,...,1.0,4.306667,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.800000
150,2.0,42.0,55.5,165.0,20.4,102.0,52.0,77.0,21.786667,14.100000,...,0.8,4.700000,212.0,68.0,63.0,135.0,11.0,16.0,33.0,5.306667
151,2.0,61.0,61.8,159.0,24.4,117.0,60.0,72.0,23.400000,15.933333,...,0.8,5.100000,235.0,66.0,56.0,166.0,15.0,23.0,60.4,5.400000


In [191]:
mse_KNN30 = mean_squared_error(data1,data_KNN30)
mse_KNN30 

31.866440260544103

### MICE

In [192]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn import svm

#### LinearRegression model

In [193]:
t_start = time.time()
lr = LinearRegression()
imp = IterativeImputer(estimator= lr , max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean')
data_LR30 = imp.fit_transform(dt30.copy())
t_end = time.time()
t_LR30 = (t_end- t_start)

[IterativeImputer] Completing matrix with shape (153, 26)
[IterativeImputer] Ending imputation round 1/10, elapsed time 0.04
[IterativeImputer] Change: 32.614381572128636, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 2/10, elapsed time 0.07
[IterativeImputer] Change: 0.33560095090956565, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 3/10, elapsed time 0.10
[IterativeImputer] Change: 0.0006708026470452211, scaled tolerance: 0.274 
[IterativeImputer] Early stopping criterion reached.


In [194]:
data_LR30 = pd.DataFrame(data_LR30)

In [195]:
data_LR30 = data_LR30.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})
data_LR30

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,57.0,47.4,153.0,20.2,114.0,75.0,69.000000,19.300000,12.609201,...,1.0,5.000000,176.0,51.0,72.0,94.0,16.0,16.0,62.00000,5.100000
1,2.0,24.0,53.2,157.0,21.6,103.0,69.0,77.990195,20.985385,8.304925,...,0.8,5.200000,159.0,45.0,52.0,98.0,13.0,17.0,50.00000,5.400000
2,2.0,62.0,54.4,165.0,20.0,95.0,75.0,83.000000,21.600000,17.548756,...,0.7,4.100000,230.0,94.0,61.0,151.0,18.0,31.0,76.00000,5.600000
3,2.0,35.0,56.3,161.0,21.7,95.0,57.0,72.000000,22.500000,7.634810,...,0.6,5.000000,198.0,47.0,68.0,120.0,16.0,20.0,35.00000,5.649574
4,1.0,46.0,72.3,170.0,25.0,133.0,77.0,70.200669,31.801752,18.804900,...,0.9,6.900000,194.0,63.0,60.0,122.0,15.0,19.0,69.00000,5.641412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1.0,60.0,72.0,170.0,24.9,131.0,74.0,69.000000,32.941031,13.800000,...,0.9,6.900000,182.0,179.0,39.0,107.0,36.0,31.0,79.00000,5.676937
149,1.0,58.0,62.0,168.0,22.0,103.0,76.0,72.000000,26.427399,10.571527,...,1.0,4.889005,191.0,62.0,47.0,132.0,16.0,33.0,42.00000,6.800000
150,2.0,42.0,55.5,165.0,20.4,102.0,52.0,77.000000,21.788612,14.100000,...,0.8,4.700000,212.0,68.0,63.0,135.0,11.0,16.0,33.00000,5.633919
151,2.0,61.0,61.8,159.0,24.4,117.0,60.0,72.000000,23.400000,18.910000,...,0.8,5.100000,235.0,66.0,56.0,166.0,15.0,23.0,53.97814,5.400000


In [196]:
mse_LR30 = mean_squared_error(data1,data_LR10)
mse_LR30 

2.246907836639976

#### Support Vector Machine model

In [197]:
t_start = time.time()
regr = svm.SVR()
svm_svr = IterativeImputer(estimator= regr, max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean')
data_SVM30 = svm_svr.fit_transform(dt30.copy())
t_end = time.time()
t_SVM30 = (t_end- t_start)

[IterativeImputer] Completing matrix with shape (153, 26)
[IterativeImputer] Ending imputation round 1/10, elapsed time 0.07
[IterativeImputer] Change: 26.16082812948214, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 2/10, elapsed time 0.14
[IterativeImputer] Change: 0.41456020862148746, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 3/10, elapsed time 0.22
[IterativeImputer] Change: 0.0014845242322566454, scaled tolerance: 0.274 
[IterativeImputer] Early stopping criterion reached.


In [198]:
data_SVM30 = pd.DataFrame(data_SVM30)

In [199]:
data_SVM30 = data_SVM30.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})
data_SVM30

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,57.0,47.4,153.0,20.2,114.0,75.0,69.000000,19.300000,13.695237,...,1.0,5.000000,176.0,51.0,72.0,94.0,16.0,16.0,62.000000,5.100000
1,2.0,24.0,53.2,157.0,21.6,103.0,69.0,74.271541,20.805937,13.686086,...,0.8,5.200000,159.0,45.0,52.0,98.0,13.0,17.0,50.000000,5.400000
2,2.0,62.0,54.4,165.0,20.0,95.0,75.0,83.000000,21.600000,14.254621,...,0.7,4.100000,230.0,94.0,61.0,151.0,18.0,31.0,76.000000,5.600000
3,2.0,35.0,56.3,161.0,21.7,95.0,57.0,72.000000,22.500000,13.833193,...,0.6,5.000000,198.0,47.0,68.0,120.0,16.0,20.0,35.000000,5.445674
4,1.0,46.0,72.3,170.0,25.0,133.0,77.0,73.800897,21.317988,14.080892,...,0.9,6.900000,194.0,63.0,60.0,122.0,15.0,19.0,69.000000,5.679860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1.0,60.0,72.0,170.0,24.9,131.0,74.0,69.000000,20.704577,13.800000,...,0.9,6.900000,182.0,179.0,39.0,107.0,36.0,31.0,79.000000,6.448011
149,1.0,58.0,62.0,168.0,22.0,103.0,76.0,72.000000,21.377651,14.028210,...,1.0,4.884489,191.0,62.0,47.0,132.0,16.0,33.0,42.000000,6.800000
150,2.0,42.0,55.5,165.0,20.4,102.0,52.0,77.000000,21.428598,14.100000,...,0.8,4.700000,212.0,68.0,63.0,135.0,11.0,16.0,33.000000,5.551322
151,2.0,61.0,61.8,159.0,24.4,117.0,60.0,72.000000,23.400000,14.255880,...,0.8,5.100000,235.0,66.0,56.0,166.0,15.0,23.0,52.268823,5.400000


In [200]:
mse_SVM30 = mean_squared_error(data1,data_SVM30)
mse_SVM30 

48.53417670400441

### EM

In [201]:
!pip install impyute
import impyute as impy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [202]:
t_start = time.time()
data_EM30 = impy.em(dt10.values.copy())
t_end = time.time()
t_EM30 = (t_end- t_start)

In [203]:
data_EM30 = pd.DataFrame(data_EM30)

In [204]:
data_EM30 = data_EM30.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})
data_EM30

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,57.0,47.4,153.0,20.200000,114.0,75.0,69.0,20.144720,12.500000,...,1.0,5.000000,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.100000
1,2.0,24.0,53.2,157.0,21.600000,103.0,69.0,78.0,22.116151,5.555094,...,0.8,5.200000,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.400000
2,2.0,62.0,54.4,165.0,20.000000,95.0,75.0,83.0,21.600000,15.200000,...,0.7,4.100000,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.600000
3,2.0,35.0,56.3,161.0,21.700000,95.0,57.0,72.0,22.500000,15.400000,...,0.6,5.000000,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.100000
4,1.0,46.0,72.3,170.0,22.790828,133.0,77.0,107.0,24.130524,19.000000,...,0.9,6.900000,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1.0,60.0,72.0,170.0,24.900000,131.0,74.0,69.0,30.500000,13.800000,...,0.9,6.900000,182.0,179.0,39.0,107.0,36.0,31.0,79.0,7.475173
149,1.0,58.0,62.0,168.0,22.000000,103.0,76.0,72.0,27.800000,11.900000,...,1.0,6.800000,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.800000
150,2.0,42.0,55.5,165.0,20.400000,102.0,52.0,77.0,22.900000,14.100000,...,0.8,4.700000,212.0,68.0,63.0,135.0,11.0,16.0,33.0,5.568363
151,2.0,61.0,61.8,159.0,24.400000,117.0,60.0,72.0,23.400000,19.200000,...,0.8,5.100000,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.498001


In [205]:
mse_EM30 = mean_squared_error(data1,data_EM30)
mse_EM30 

21.695245611831062

# MSE

In [206]:
mse_value = {'Method':['MEAN','Miss Forest','KNN','MICE-LR','MICE-SVM','EM'],'10%':[mse_MEAN10 ,mse_MF10 ,mse_KNN10 ,mse_LR10 ,mse_SVM10 ,mse_EM10 ],
      '20%':[mse_MEAN20 ,mse_MF20 ,mse_KNN20 ,mse_LR20 ,mse_SVM20,mse_EM20 ],'30%':[mse_MEAN30 ,mse_MF30 ,mse_KNN30 ,mse_LR30 ,mse_SVM30 ,mse_EM30 ]}

In [207]:
data_MSE = pd.DataFrame.from_dict(mse_value)
data_MSE

,Method,10%,20%,30%
0,MEAN,14.794385,21.640901,14.794385
1,Miss Forest,2.314475,6.124488,13.375977
2,KNN,10.106348,16.685132,31.866440
3,MICE-LR,2.246908,7.414225,2.246908
4,MICE-SVM,15.029068,23.465243,48.534177
5,EM,25.211814,40.297898,21.695246
